In [1]:
import pandas as pd
import numpy as np
import os
import re
from datetime import datetime


def getData(path,personName):
	responseDictionary = dict()
	fbFile = open(path, 'r') 
	allLines = fbFile.readlines()
	myMessage, otherPersonsMessage, currentSpeaker = "","",""
	for index,lines in enumerate(allLines):
	    rightBracket = lines.find(']') + 2
	    justMessage = lines[rightBracket:]
	    colon = justMessage.find(':')
	    # Find messages that I sent
	    if (justMessage[:colon] == personName):
	        if not myMessage:
	            # Want to find the first message that I send (if I send multiple in a row)
	            startMessageIndex = index - 1
	        myMessage += justMessage[colon+2:]
	        
	    elif myMessage:
	        # Now go and see what message the other person sent by looking at previous messages
	        for counter in range(startMessageIndex, 0, -1):
	            currentLine = allLines[counter]
	            rightBracket = currentLine.find(']') + 2
	            justMessage = currentLine[rightBracket:]
	            colon = justMessage.find(':')
	            if not currentSpeaker:
	                # The first speaker not named me
	                currentSpeaker = justMessage[:colon]
	            elif (currentSpeaker != justMessage[:colon] and otherPersonsMessage):
	                # A different person started speaking, so now I know that the first person's message is done
	                otherPersonsMessage = cleanMessage(otherPersonsMessage)
	                myMessage = cleanMessage(myMessage)
	                responseDictionary[otherPersonsMessage] = myMessage
	                break
	            otherPersonsMessage = justMessage[colon+2:] + otherPersonsMessage
	        myMessage, otherPersonsMessage, currentSpeaker = "","",""    
	return responseDictionary


def cleanMessage(message):
	# Remove new lines within message
	cleanedMessage = message.replace('\n',' ').lower()
	# Deal with some weird tokens
	cleanedMessage = cleanedMessage.replace("\xc2\xa0", "")
	# Remove punctuation
	cleanedMessage = re.sub('([.,!?])','', cleanedMessage)
	# Remove multiple spaces in message
	cleanedMessage = re.sub(' +',' ', cleanedMessage)
	return cleanedMessage

In [2]:
personName = raw_input('Enter your full name: ')

Enter your full name: Mahmoud Ashraf Khairy


In [3]:
personName

'Mahmoud Ashraf Khairy'

In [4]:
FacebookDictionary = {}
FacebookDictionary.update(getData('fbmessages.txt', personName))

In [5]:
FacebookDictionary

{'ok lemme know if she does ': "she still didn't answer lol ",
 'no i dont at all that look the same ill see how it looks ill go shopping on thursday and in montreal theres a shopping mall right next to our hotel ill check there too ': 'it doesnt look good ',
 "http://wwwfacebookcom/#/photophpfbid=10150601491482497&set=a16204695249611537763843132496&type=1&theater :'( 7asbanallah wan3mal wakiil ": 'ya3ny nezelty wala l2a 3shan ana mafhemtesh :/ ',
 'ok ty babe ': 'do u miss ur gf ur crush ',
 'yeaaa thats even here in waterloo and urrr da only africann ': 'yaaaaa whats good ma niggas ',
 'ps -- ': 'aah :d wenty bete3mely eeh ',
 'you cant transfer ': 'it takes a week to get from egypt i got 1 2 3 4 5 6 7 8 mill in my bank account ',
 'thank you\xe2\x9d\xa4\xef\xb8\x8f i love you\xe2\x9d\xa4\xef\xb8\x8f ': 'love u more\xe2\x9d\xa4\xef\xb8\x8f\xe2\x9d\xa4\xef\xb8\x8f ',
 "you're busyyy nooo ": "i'm not ",
 'yo are we going to work on 146 tomorrow sorry i missed last time i felt so sick i

In [6]:
print 'Total len of dictionary', len(FacebookDictionary)

Total len of dictionary 56603


In [7]:
import numpy as np
print 'Saving conversation data dictionary'
np.save('conversationDictionary.npy', FacebookDictionary)

Saving conversation data dictionary


In [8]:
conversationFile = open('conversationData.txt', 'w')
for key,value in FacebookDictionary.iteritems():
	if (not key.strip() or not value.strip()):
		# If there are empty strings
		continue
   	conversationFile.write(key.strip() + value.strip())

In [9]:
import tensorflow as tf 
import numpy as np 
import re
from collections import Counter
import sys
import math
from random import randint
import pickle
import os

In [10]:
def processDataset(filename):
	openedFile = open(filename, 'r')
	allLines = openedFile.readlines()
	myStr = ""
	for line in allLines:
	    myStr += line
	finalDict = Counter(myStr.split())
	return myStr, finalDict

In [ ]:
def MyprocessDataset(filename):
    readFile = open(filename, 'r') # 'r' means read file
    myLines = readFile.readlines()
    myStr = ""
    for Line in myLines:
        myStr += Line
    finalDict = Counter(myStr.split())
    return myStr, finalDict

In [11]:
fullCorpus, datasetDictionary = processDataset('conversationData.txt')
print 'Finished parsing and cleaning dataset'

wordList = datasetDictionary.keys()

with open("wordList.txt", "wb") as fp: 
    pickle.dump(wordList, fp)


Finished parsing and cleaning dataset


In [12]:
import tensorflow as tf 
import numpy as np 
import sys
from random import randint
import datetime
from sklearn.utils import shuffle
import pickle
import os

In [13]:
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'

In [14]:
# Hyperparamters
batchSize = 24
maxEncoderLength = 15
maxDecoderLength = maxEncoderLength
lstmUnits = 112
embeddingDim = lstmUnits
numLayersLSTM = 3
numIterations = 500000

In [15]:
with open("wordList.txt", "rb") as fp:
	wordList = pickle.load(fp)

In [16]:
wordList

['studyyyyyynothing7t',
 'beautifulbelieve',
 'http://wwwfacebookcom/photophpfbid=179094782194340&set=a15600308450351026253155844844519334&type=1&theater',
 'episodebaratheons',
 'peoplei\xe2\x80\x99m',
 "3'am2",
 'friendzonedi',
 'gag',
 'wannaaaa',
 'spiders',
 'hanging',
 'minthere',
 'dorha',
 "schooli'm",
 'tomisa',
 'enhrdaama',
 'jokescheck',
 'lesterdooooo',
 'lolstillmine',
 '--ya3am',
 'gab',
 'hairlier',
 ':de3meleelo',
 'opener',
 'storiesim',
 'crossbas',
 'moreee',
 "men3'er",
 'lateahhh',
 'whatfinished',
 'diy(0)=0',
 'hahahah',
 'europewhere',
 'discribed',
 'mindblownlol',
 'nightttttyywhy',
 '\xd8\xb3\xd8\xb1\xd9\x8a\xd8\xb9\xd8\xa9',
 'right"im',
 'sendingu',
 'thenpmathwhether',
 'sendingi',
 'studentsplease',
 'bringing',
 'sendingg',
 'lollolwow',
 'hasmigwhatnothing',
 "whatttit's",
 'mazoneshomaleven',
 'datinglelyes',
 'meeeedo',
 'wooden',
 '598s',
 'infinityyeamy',
 'wednesday',
 'msbtaha',
 'sa777ok',
 'themvlow',
 'elgaw',
 'ballashhh',
 'fhmtk',
 'mcdonal

In [17]:
vocabSize = len(wordList)

In [18]:
wordVecDimensions = 100

In [19]:
padVector = np.zeros((1, wordVecDimensions), dtype='int32')
EOSVector = np.ones((1, wordVecDimensions), dtype='int32')

In [20]:
wordList.append('<pad>')
wordList.append('<EOS>')
vocabSize = vocabSize + 2

In [21]:
def createTrainingMatrices(conversationFileName, wList, maxLen):
	conversationDictionary = np.load(conversationFileName).item()
	numExamples = len(conversationDictionary)
	xTrain = np.zeros((numExamples, maxLen), dtype='int32')
	yTrain = np.zeros((numExamples, maxLen), dtype='int32')
	for index,(key,value) in enumerate(conversationDictionary.iteritems()):
		# Will store integerized representation of strings here (initialized as padding)
		encoderMessage = np.full((maxLen), wList.index('<pad>'), dtype='int32')
		decoderMessage = np.full((maxLen), wList.index('<pad>'), dtype='int32')
		# Getting all the individual words in the strings
		keySplit = key.split()
		valueSplit = value.split()
		keyCount = len(keySplit)
		valueCount = len(valueSplit)
		# Throw out sequences that are too long
		if (keyCount > (maxLen - 1) or valueCount > (maxLen - 1)):
			continue
		# Integerize the encoder string
		for keyIndex, word in enumerate(keySplit):
			try:
				encoderMessage[keyIndex] = wList.index(word)
			except ValueError:
				# TODO: This isnt really the right way to handle this scenario
				encoderMessage[keyIndex] = 0
		encoderMessage[keyIndex + 1] = wList.index('<EOS>')
		# Integerize the decoder string
		for valueIndex, word in enumerate(valueSplit):
			try:
				decoderMessage[valueIndex] = wList.index(word)
			except ValueError:
				decoderMessage[valueIndex] = 0
		decoderMessage[valueIndex + 1] = wList.index('<EOS>')
		xTrain[index] = encoderMessage
		yTrain[index] = decoderMessage
	# Remove rows with all zeros
	yTrain = yTrain[~np.all(yTrain == 0, axis=1)]
	xTrain = xTrain[~np.all(xTrain == 0, axis=1)]
	numExamples = xTrain.shape[0]
	return numExamples, xTrain, yTrain


In [23]:
numTrainingExamples, xTrain, yTrain = createTrainingMatrices('conversationDictionary.npy', wordList, maxEncoderLength)
np.save('Seq2SeqXTrain.npy', xTrain)
np.save('Seq2SeqYTrain.npy', yTrain)
print 'Finished creating training matrices'

Finished creating training matrices


In [24]:
def getTrainingBatch(localXTrain, localYTrain, localBatchSize, maxLen):
	num = randint(0,numTrainingExamples - localBatchSize - 1)
	arr = localXTrain[num:num + localBatchSize]
	labels = localYTrain[num:num + localBatchSize]
	# Reversing the order of encoder string apparently helps as per 2014 paper
	reversedList = list(arr)
	for index,example in enumerate(reversedList):
		reversedList[index] = list(reversed(example))

	# Lagged labels are for the training input into the decoder
	laggedLabels = []
	EOStokenIndex = wordList.index('<EOS>')
	padTokenIndex = wordList.index('<pad>')
	for example in labels:
		eosFound = np.argwhere(example==EOStokenIndex)[0]
		shiftedExample = np.roll(example,1)
		shiftedExample[0] = EOStokenIndex
		# The EOS token was already at the end, so no need for pad
		if (eosFound != (maxLen - 1)):
			shiftedExample[eosFound+1] = padTokenIndex
		laggedLabels.append(shiftedExample)

	# Need to transpose these 
	reversedList = np.asarray(reversedList).T.tolist()
	labels = labels.T.tolist()
	laggedLabels = np.asarray(laggedLabels).T.tolist()
	return reversedList, labels, laggedLabels

def translateToSentences(inputs, wList, encoder=False):
	EOStokenIndex = wList.index('<EOS>')
	padTokenIndex = wList.index('<pad>')
	numStrings = len(inputs[0])
	numLengthOfStrings = len(inputs)
	listOfStrings = [''] * numStrings
	for mySet in inputs:
		for index,num in enumerate(mySet):
			if (num != EOStokenIndex and num != padTokenIndex):
				if (encoder):
					# Encodings are in reverse!
					listOfStrings[index] = wList[num] + " " + listOfStrings[index]
				else:
					listOfStrings[index] = listOfStrings[index] + " " + wList[num]
	listOfStrings = [string.strip() for string in listOfStrings]
	return listOfStrings

def getTestInput(inputMessage, wList, maxLen):
	encoderMessage = np.full((maxLen), wList.index('<pad>'), dtype='int32')
	inputSplit = inputMessage.lower().split()
	for index,word in enumerate(inputSplit):
		try:
			encoderMessage[index] = wList.index(word)
		except ValueError:
			continue
	encoderMessage[index + 1] = wList.index('<EOS>')
	encoderMessage = encoderMessage[::-1]
	encoderMessageList=[]
	for num in encoderMessage:
		encoderMessageList.append([num])
	return encoderMessageList

def idsToSentence(ids, wList):
    EOStokenIndex = wList.index('<EOS>')
    padTokenIndex = wList.index('<pad>')
    myStr = ""
    listOfResponses=[]
    for num in ids:
        if (num[0] == EOStokenIndex or num[0] == padTokenIndex):
            listOfResponses.append(myStr)
            myStr = ""
        else:
            myStr = myStr + wList[num[0]] + " "
    if myStr:
        listOfResponses.append(myStr)
    listOfResponses = [i for i in listOfResponses if i]
    return listOfResponses

In [25]:
tf.reset_default_graph()

# Create the placeholders
encoderInputs = [tf.placeholder(tf.int32, shape=(None,)) for i in range(maxEncoderLength)]
decoderLabels = [tf.placeholder(tf.int32, shape=(None,)) for i in range(maxDecoderLength)]
decoderInputs = [tf.placeholder(tf.int32, shape=(None,)) for i in range(maxDecoderLength)]
feedPrevious = tf.placeholder(tf.bool)

In [26]:
encoderLSTM = tf.contrib.rnn.BasicLSTMCell(lstmUnits, state_is_tuple=True)

In [27]:
decoderOutputs, decoderFinalState = tf.contrib.legacy_seq2seq.embedding_rnn_seq2seq(encoderInputs, 
                                                                                    decoderInputs, 
                                                                                    encoderLSTM, 
                                                                                    vocabSize, 
                                                                                    vocabSize, 
                                                                                    embeddingDim, 
                                                                                    feed_previous=feedPrevious)

In [28]:
decoderPrediction = tf.argmax(decoderOutputs, 2)

In [29]:
lossWeights = [tf.ones_like(l, dtype=tf.float32) for l in decoderLabels]
loss = tf.contrib.legacy_seq2seq.sequence_loss(decoderOutputs, decoderLabels, lossWeights, vocabSize)
optimizer = tf.train.AdamOptimizer(1e-4).minimize(loss)

In [30]:
sess = tf.Session()
saver = tf.train.Saver()
# If you're loading in a saved model, use the following
#saver.restore(sess, tf.train.latest_checkpoint('models/'))
sess.run(tf.global_variables_initializer())

In [31]:
# Uploading results to Tensorboard
tf.summary.scalar('Loss', loss)
merged = tf.summary.merge_all()
logdir = "tensorboard/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S") + "/"
writer = tf.summary.FileWriter(logdir, sess.graph)

In [32]:
encoderTestStrings = ["whats up bro",
                      "hi",
                      "hey how are you",
                      "eh",
                      "i love you"
                     ]

In [33]:
zeroVector = np.zeros((1), dtype='int32')

In [34]:
for i in range(numIterations):

	encoderTrain, decoderTargetTrain, decoderInputTrain = getTrainingBatch(xTrain, yTrain, batchSize, maxEncoderLength)
	feedDict = {encoderInputs[t]: encoderTrain[t] for t in range(maxEncoderLength)}
	feedDict.update({decoderLabels[t]: decoderTargetTrain[t] for t in range(maxDecoderLength)})
	feedDict.update({decoderInputs[t]: decoderInputTrain[t] for t in range(maxDecoderLength)})
	feedDict.update({feedPrevious: False})

	curLoss, _, pred = sess.run([loss, optimizer, decoderPrediction], feed_dict=feedDict)
	
	if (i % 50 == 0):
		print('Current loss:', curLoss, 'at iteration', i)
		summary = sess.run(merged, feed_dict=feedDict)
		writer.add_summary(summary, i)
	if (i % 25 == 0 and i != 0):
		num = randint(0,len(encoderTestStrings) - 1)
		print encoderTestStrings[num]
		inputVector = getTestInput(encoderTestStrings[num], wordList, maxEncoderLength);
		feedDict = {encoderInputs[t]: inputVector[t] for t in range(maxEncoderLength)}
		feedDict.update({decoderLabels[t]: zeroVector for t in range(maxDecoderLength)})
		feedDict.update({decoderInputs[t]: zeroVector for t in range(maxDecoderLength)})
		feedDict.update({feedPrevious: True})
		ids = (sess.run(decoderPrediction, feed_dict=feedDict))
		print idsToSentence(ids, wordList)

	if (i % 10000 == 0 and i != 0):
		savePath = saver.save(sess, "pretrained_seq2seq_new.ckpt", global_step=i)

('Current loss:', 11.792348, 'at iteration', 0)
i love you
[]
('Current loss:', 11.57716, 'at iteration', 50)
hi
[]
hey how are you
[]
('Current loss:', 10.117448, 'at iteration', 100)
hey how are you
[]
i love you
[]
('Current loss:', 8.3583117, 'at iteration', 150)
i love you
[]
whats up bro
[]
('Current loss:', 6.9422603, 'at iteration', 200)
whats up bro
[]
i love you
[]
('Current loss:', 5.9378028, 'at iteration', 250)
hey how are you
[]
i love you
[]
('Current loss:', 5.6092486, 'at iteration', 300)
hi
[]
hi
[]
('Current loss:', 4.7099671, 'at iteration', 350)
whats up bro
[]
whats up bro
[]
('Current loss:', 4.6967797, 'at iteration', 400)
whats up bro
[]
hi
[]
('Current loss:', 3.9221287, 'at iteration', 450)
i love you
[]
hi
[]
('Current loss:', 3.3816307, 'at iteration', 500)
eh
[]
hi
[]
('Current loss:', 3.6397047, 'at iteration', 550)
hey how are you
[]
hi
[]
('Current loss:', 3.7004254, 'at iteration', 600)
whats up bro
[]
hi
[]
('Current loss:', 3.3836544, 'at iteration',

['i ']
i love you
['i ']
('Current loss:', 2.6750174, 'at iteration', 5450)
eh
['studyyyyyynothing7t ']
i love you
['i ']
('Current loss:', 2.2419965, 'at iteration', 5500)
i love you
['i ']
whats up bro
['i ']
('Current loss:', 2.6137712, 'at iteration', 5550)
hey how are you
['i i ']
hi
['i ']
('Current loss:', 2.9715884, 'at iteration', 5600)
eh
['studyyyyyynothing7t ']
whats up bro
['i ']
('Current loss:', 2.854564, 'at iteration', 5650)
i love you
['i i ']
whats up bro
['i ']
('Current loss:', 2.9975364, 'at iteration', 5700)
hi
['i ']
hey how are you
['i ']
('Current loss:', 3.5855446, 'at iteration', 5750)
hi
['i ']
whats up bro
['i ']
('Current loss:', 2.6572368, 'at iteration', 5800)
i love you
['i ']
hey how are you
['i ']
('Current loss:', 2.6670597, 'at iteration', 5850)
i love you
['i i ']
eh
['studyyyyyynothing7t ']
('Current loss:', 2.8245268, 'at iteration', 5900)
hey how are you
['i ']
i love you
['i u ']
('Current loss:', 2.0507774, 'at iteration', 5950)
whats up bro


('Current loss:', 3.2116477, 'at iteration', 8550)
hey how are you
['i the the a a studyyyyyynothing7t ']
whats up bro
['i the the a a el el el el el el el el el el ']
('Current loss:', 2.7323663, 'at iteration', 8600)
whats up bro
['i the the a a el el el el el el el el el el ']
eh
['studyyyyyynothing7t ']
('Current loss:', 2.9806812, 'at iteration', 8650)
hi
['studyyyyyynothing7t ']
whats up bro
['i the the a a el el el el el el el el el el ']
('Current loss:', 2.4300549, 'at iteration', 8700)
i love you
['i the the a a studyyyyyynothing7t ']
eh
['studyyyyyynothing7t ']
('Current loss:', 2.8467624, 'at iteration', 8750)
i love you
['i the the the the a studyyyyyynothing7t ']
i love you
['i the the the the the a studyyyyyynothing7t ']
('Current loss:', 2.8144958, 'at iteration', 8800)
hey how are you
['i the the the the the the studyyyyyynothing7t ']
eh
['studyyyyyynothing7t ']
('Current loss:', 2.5651782, 'at iteration', 8850)
hey how are you
['i the the the the the a el el el el el 

['i not ']
hey how are you
['i not ']
('Current loss:', 2.3334451, 'at iteration', 11950)
hey how are you
['i not ']
i love you
['i not ']
('Current loss:', 2.1573579, 'at iteration', 12000)
eh
['studyyyyyynothing7t ']
eh
['studyyyyyynothing7t ']
('Current loss:', 2.4499009, 'at iteration', 12050)
whats up bro
['studyyyyyynothing7t ']
i love you
['i not ']
('Current loss:', 2.6878328, 'at iteration', 12100)
hi
['studyyyyyynothing7t ']
whats up bro
['studyyyyyynothing7t ']
('Current loss:', 2.4101334, 'at iteration', 12150)
i love you
["i don't the the the the the the the the the the the the the "]
hey how are you
["i don't the the the the the the the the the the the the the "]
('Current loss:', 3.0684793, 'at iteration', 12200)
hi
['studyyyyyynothing7t ']
whats up bro
['studyyyyyynothing7t ']
('Current loss:', 2.4437466, 'at iteration', 12250)
hi
['studyyyyyynothing7t ']
hi
['studyyyyyynothing7t ']
('Current loss:', 2.4995048, 'at iteration', 12300)
hey how are you
["i don't the the a 

['studyyyyyynothing7t ']
i love you
["i don't have the call "]
('Current loss:', 2.8724625, 'at iteration', 15250)
i love you
["i don't have the call "]
i love you
["i don't have the call "]
('Current loss:', 2.3201685, 'at iteration', 15300)
i love you
["i don't have the call "]
hey how are you
["i don't have the call "]
('Current loss:', 2.482239, 'at iteration', 15350)
whats up bro
['studyyyyyynothing7t ']
eh
['studyyyyyynothing7t ']
('Current loss:', 2.7651482, 'at iteration', 15400)
eh
['studyyyyyynothing7t ']
i love you
["i don't have the call "]
('Current loss:', 1.9744358, 'at iteration', 15450)
hey how are you
["i don't have the call "]
whats up bro
['studyyyyyynothing7t ']
('Current loss:', 2.3757551, 'at iteration', 15500)
i love you
["i don't have the call "]
eh
['studyyyyyynothing7t ']
('Current loss:', 2.7087021, 'at iteration', 15550)
eh
['studyyyyyynothing7t ']
whats up bro
['studyyyyyynothing7t ']
('Current loss:', 2.2240136, 'at iteration', 15600)
hi
['studyyyyyynothi

...

["i don't want to ", 'is a lot of the same ']
i love you
["i don't want to do ", 'is a lot of ']
('Current loss:', 1.7111782, 'at iteration', 92300)
hi
['studyyyyyynothing7t ']
hey how are you
["i don't want to ", 'is a lot of the same ']
('Current loss:', 2.0505397, 'at iteration', 92350)
hey how are you
["i don't want to ", 'is a lot of the same ']
whats up bro
['studyyyyyynothing7t ', 'is the exam is a lot of the world ']
('Current loss:', 2.1282721, 'at iteration', 92400)
whats up bro
['studyyyyyynothing7t ', 'is the exam is a lot of the world ']
eh
['studyyyyyynothing7t ', 'keda gedan ']
('Current loss:', 1.8569998, 'at iteration', 92450)
hey how are you
["i don't want to ", 'is a lot of the same ']
eh
['studyyyyyynothing7t ', 'keda ']
('Current loss:', 1.5163479, 'at iteration', 92500)
whats up bro
['studyyyyyynothing7t ', 'is the exam is a lot of the world ']
hi
['studyyyyyynothing7t ']
('Current loss:', 2.0513661, 'at iteration', 92550)
eh
['studyyyyyynothing7t ', 'keda ']
what

['studyyyyyynothing7t ']
hi
['studyyyyyynothing7t ']
('Current loss:', 1.5371699, 'at iteration', 94850)
whats up bro
['studyyyyyynothing7t ', 'is the exam is a lot of the world ']
hey how are you
["i don't want to ", 'is a lot of the same ']
('Current loss:', 1.6762389, 'at iteration', 94900)
eh
['studyyyyyynothing7t ', 'keda keda neik ']
eh
['studyyyyyynothing7t ', 'keda neik :d ']
('Current loss:', 1.3100452, 'at iteration', 94950)
whats up bro
['studyyyyyynothing7t ', 'is the exam is a lot of the world ']
i love you
["i don't want to do ", 'is a lot of ']
('Current loss:', 1.5325246, 'at iteration', 95000)
whats up bro
['studyyyyyynothing7t ', 'is the exam is a lot of the world ']
hi
['studyyyyyynothing7t ']
('Current loss:', 1.7192711, 'at iteration', 95050)
hey how are you
["i don't want to do ", 'is a lot of the same ']
i love you
["i don't want to do ", 'is a lot of ']
('Current loss:', 1.8142503, 'at iteration', 95100)
eh
['studyyyyyynothing7t ', 'keda neik :d ']
hi
['studyyyy

['studyyyyyynothing7t ', 'keda neik ']
hey how are you
["i don't want to ", 'is a lot of the same ']
('Current loss:', 1.569767, 'at iteration', 97500)
hey how are you
["i don't want to ", 'is a lot of the same ']
i love you
["i don't want to ", 'is a lot of the same ']
('Current loss:', 2.2013175, 'at iteration', 97550)
eh
['studyyyyyynothing7t ', 'keda neik :d ']
whats up bro
['studyyyyyynothing7t ', 'is the same of the world ', 'is like ']
('Current loss:', 1.6387491, 'at iteration', 97600)
hey how are you
["i don't want to ", 'is a lot of the same ']
hey how are you
["i don't want to ", 'is a lot of the same ']
('Current loss:', 1.2038842, 'at iteration', 97650)
whats up bro
['studyyyyyynothing7t ', 'is the same of the world ']
eh
['studyyyyyynothing7t ', 'keda ']
('Current loss:', 1.6186496, 'at iteration', 97700)
whats up bro
['studyyyyyynothing7t ', 'is the same of the world ']
eh
['studyyyyyynothing7t ', 'keda neik :d ']
('Current loss:', 1.7169561, 'at iteration', 97750)
i lov

hey how are you
["i don't want to ", 'is a lot of the same ']
('Current loss:', 1.627694, 'at iteration', 100150)
hi
['studyyyyyynothing7t ']
whats up bro
['studyyyyyynothing7t ', 'is the exam is a lot of the world ']
('Current loss:', 1.4715137, 'at iteration', 100200)
hi
['studyyyyyynothing7t ']
hey how are you
["i don't want to ", 'is a lot of the same ']
('Current loss:', 1.8721842, 'at iteration', 100250)
hi
['studyyyyyynothing7t ']
hey how are you
["i don't want to ", 'to u ', 'is a email ']
('Current loss:', 1.5414333, 'at iteration', 100300)
whats up bro
['studyyyyyynothing7t ', 'is the exam is a lot of the world ']
eh
['studyyyyyynothing7t el a5bar el studyyyyyynothing7t ']
('Current loss:', 1.9477507, 'at iteration', 100350)
hi
['studyyyyyynothing7t ']
eh
['studyyyyyynothing7t ', 'keda neik ']
('Current loss:', 1.29482, 'at iteration', 100400)
hi
['studyyyyyynothing7t ']
eh
['studyyyyyynothing7t ', 'keda neik ']
('Current loss:', 1.5842768, 'at iteration', 100450)
whats up br

KeyboardInterrupt: 

In [33]:
savePath = saver.save(sess, "pretrained_seq2seq.ckpt", global_step=i)